In [1]:
# load libraries
%matplotlib inline
import pandas as pd
import numpy as np
import time
import os

In [2]:
#read in the bag of words CSV as a Pandas DataFrame
foo_df = pd.read_csv("bagofwords_draft3.csv", header=0)
#show the first five lines to verify upload
df = foo_df.replace(np.nan, '', regex=True)
df.sort_values("FirstName")
df.sort_values("LastName")

,CourseTaken,CourseNum,CourseName,CourseDescription,YearTaken,FirstName,LastName,MatYear,Gender,Background,...,global,designs,artificial,intelligence,delivery,speakers,oral,discussions,web,healthy
762,1,15.375,Development Ventures,"Seminar on founding, financing, and building e...",2019,Saket,Adhikarla,2019,M,Design,...,0,0,0,0,0,0,0,0,0,0
764,1,15.846,Branding,"Provides a foundation for building, managing, ...",2020,Saket,Adhikarla,2019,M,Design,...,0,0,0,0,0,0,0,0,0,0
763,1,15.665,Power and Negotiation,Provides understanding of the theory and proce...,2019,Saket,Adhikarla,2019,M,Design,...,0,0,0,0,0,0,0,0,0,0
765,1,15.915,Lab for Sustainable Business,"Develops a pragmatic, action-oriented approach...",2020,Saket,Adhikarla,2019,M,Design,...,0,0,0,0,0,0,0,1,0,3
766,1,16.855,Sys Architecting: Enterprises,Focuses on principles and practices for archit...,2020,Saket,Adhikarla,2019,M,Design,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
861,1,16.453,Human Sys Engineering,Provides a fundamental understanding of human ...,2019,Tianqi,Zhou,2018,F,Design,...,0,0,0,0,0,0,1,0,0,0
862,1,21G.219,Academic & Prof Writing ELS,Writing module for high- intermediate ELS stud...,2018,Tianqi,Zhou,2018,F,Design,...,0,0,0,0,0,0,0,0,0,0
863,1,6.835,Intell Multi User Interfaces,Implementation and evaluation of intelligent m...,2019,Tianqi,Zhou,2018,F,Design,...,0,0,0,0,0,0,0,0,0,0
866,1,6.894,Adv Top Graph Human Comp Inter,Advanced study of topics in graphics and human...,2020,Tianqi,Zhou,2018,F,Design,...,0,0,0,0,0,0,0,0,0,0


In [3]:
#upload the update list of courses to use
check_df = pd.read_csv("IDM_Course_Numbers.csv", header=0)
#show the first five lines to verify upload
check_df

,Course_Num,Course_Name
0,1.001,Eng Computation & Data Science
1,1.125,Arch& Engineering Software Sys
2,1.146,Engr Sys Analysis for Design
3,1.151,Probability & Stats in Eng
4,1.26,Logistics Systems
...,...,...
183,MAS.600,Human 2.0
184,MAS.712,Learning Creative Learning
185,MAS.834,Tangible Interfaces
186,MAS.836,Sens Tech Interact Environs


In [4]:
counter = 0
for i in df.index:
    if df.at[i,"CourseNum"] not in check_df["Course_Num"].unique():
        counter += 1
print(counter)

66


In [5]:
#Initialize the output dataframe with a dummy row
init_data = [list(df.columns)]
output_df = pd.DataFrame(init_data, columns = list(df.columns))
#Verify the new dataframe looks as intended
output_df.head()

,CourseTaken,CourseNum,CourseName,CourseDescription,YearTaken,FirstName,LastName,MatYear,Gender,Background,...,global,designs,artificial,intelligence,delivery,speakers,oral,discussions,web,healthy
0,CourseTaken,CourseNum,CourseName,CourseDescription,YearTaken,FirstName,LastName,MatYear,Gender,Background,...,global,designs,artificial,intelligence,delivery,speakers,oral,discussions,web,healthy


In [6]:
#Output dataframe index tracker
k=0

#track which students have been done so as not to duplicate work
student_tracker = []

#for each row in the dataframe
for i in df.index:
    
    #if the course is not in the updated course list, skip it
    if df.at[i,"CourseNum"] not in check_df["Course_Num"].unique():
        continue
    
    #pull in the student's data
    yr_taken = df.at[i,"YearTaken"]
    first = df.at[i,"FirstName"]
    last = df.at[i,"LastName"]
    yr_mat = df.at[i,"MatYear"]
    gender = df.at[i,"Gender"]
    background = df.at[i,"Background"]
    prev = df.at[i,"PreviousRole"]
    yrs_worked = df.at[i,"NbrYearsWorked"]
    current = df.at[i,"CurrentRole"]
        
    #check if we've already done this student
    already_done = False
    for student in student_tracker:
        if (student[0] == first) and (student[1] == last):
            already_done = True
            break
    if already_done:
        continue
    else:
        student_tracker.append((first, last))
        
    #track which courses we've added to avoid duplicates
    new_courses = []
        
    #Iterate through all of the other rows to look for classes taken
    for j in range(i, len(df.index)):
        
        #if the course is not in the updated course list, skip it
        if df.at[j,"CourseNum"] not in check_df["Course_Num"].unique():
            continue
        
        #check if we've already added it
        elif df.at[j,"CourseNum"] in new_courses:
            continue
            
        #each student only takes a given class once, so if we find their name we can add it directly to the output
        elif (df.at[j,"FirstName"] == first) and (df.at[j,"LastName"] == last):
            new_courses.append(df.at[j,"CourseNum"])
            temp_data1 = [list(df.iloc[j])]
            temp_df1 = pd.DataFrame(temp_data1, columns = list(df.columns))
            output_df.loc[k] = temp_df1.loc[0]
            k += 1
            
    #Iterate through all of the other rows to look for classes not taken
    for j in range(i, len(df.index)):
        
        #if the course is not in the updated course list, skip it
        if df.at[j,"CourseNum"] not in check_df["Course_Num"].unique():
            continue
        
        #if we've already added it, skip it
        elif df.at[j,"CourseNum"] in new_courses:
            continue
            
        #if we haven't added it already, make a new row in output with the correct student info
        else:
            new_courses.append(df.at[j,"CourseNum"])
            temp_data2 = [list(df.iloc[j])]
            temp_df2 = pd.DataFrame(temp_data2, columns = list(df.columns))
            temp_df2.at[0,"CourseTaken"] = 0
            temp_df2.at[0,"YearTaken"] = yr_taken
            temp_df2.at[0,"FirstName"] = first
            temp_df2.at[0,"LastName"] = last
            temp_df2.at[0,"MatYear"] = yr_mat
            temp_df2.at[0,"Gender"] = gender
            temp_df2.at[0,"Background"] = background
            temp_df2.at[0,"PreviousRole"] = prev
            temp_df2.at[0,"NbrYearsWorked"] = yrs_worked
            temp_df2.at[0,"CurrentRole"] = current
            output_df.loc[k] = temp_df2.loc[0]
            k += 1
            
output_df.head()

,CourseTaken,CourseNum,CourseName,CourseDescription,YearTaken,FirstName,LastName,MatYear,Gender,Background,...,global,designs,artificial,intelligence,delivery,speakers,oral,discussions,web,healthy
0,1,15.232,Bus Models in Frontier Mkts,"Examines how new approaches to operations, rev...",2019,Aaron,Stinnett,2018,M,Business,...,0,0,0,0,1,1,0,0,0,0
1,1,15.518,Taxes and Business Strategy,Provides conceptual framework for thinking abo...,2020,Aaron,Stinnett,2018,M,Business,...,0,0,0,0,0,0,0,1,0,0
2,1,15.761,Intro Operations Management,"Imparts concepts, techniques, and tools to des...",2020,Aaron,Stinnett,2018,M,Business,...,0,0,0,0,0,0,0,0,0,0
3,1,15.915,Lab for Sustainable Business,"Develops a pragmatic, action-oriented approach...",2019,Aaron,Stinnett,2018,M,Business,...,0,0,0,0,0,0,0,1,0,3
4,1,16.355,Engineering of Software,Reading and discussion on issues in the engine...,2020,Aaron,Stinnett,2018,M,Business,...,0,0,0,0,0,0,0,0,0,0


In [7]:
#Save it as a CSV to the local directory
output_df.to_csv(r'draft8_full.csv', index = False)

In [8]:
output_df

,CourseTaken,CourseNum,CourseName,CourseDescription,YearTaken,FirstName,LastName,MatYear,Gender,Background,...,global,designs,artificial,intelligence,delivery,speakers,oral,discussions,web,healthy
0,1,15.232,Bus Models in Frontier Mkts,"Examines how new approaches to operations, rev...",2019,Aaron,Stinnett,2018,M,Business,...,0,0,0,0,1,1,0,0,0,0
1,1,15.518,Taxes and Business Strategy,Provides conceptual framework for thinking abo...,2020,Aaron,Stinnett,2018,M,Business,...,0,0,0,0,0,0,0,1,0,0
2,1,15.761,Intro Operations Management,"Imparts concepts, techniques, and tools to des...",2020,Aaron,Stinnett,2018,M,Business,...,0,0,0,0,0,0,0,0,0,0
3,1,15.915,Lab for Sustainable Business,"Develops a pragmatic, action-oriented approach...",2019,Aaron,Stinnett,2018,M,Business,...,0,0,0,0,0,0,0,1,0,3
4,1,16.355,Engineering of Software,Reading and discussion on issues in the engine...,2020,Aaron,Stinnett,2018,M,Business,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12766,1,15.358,Platform Strategy & Entrepren,Considers key strategic concepts and ideas use...,2020,Zahra,Kanji,2019,F,Engineering,...,0,0,0,0,0,0,0,0,0,0
12767,1,15.401,Managerial Finance,Introduction to finance from the perspective o...,2019,Zahra,Kanji,2019,F,Engineering,...,0,0,0,0,0,0,0,0,0,0
12768,1,15.761,Intro Operations Management,"Imparts concepts, techniques, and tools to des...",2020,Zahra,Kanji,2019,F,Engineering,...,0,0,0,0,0,0,0,0,0,0
12769,1,2.778,Large & Complex Systems Design,Examines structured principles and processes t...,2019,Zahra,Kanji,2019,F,Engineering,...,0,0,0,0,0,0,0,0,0,0


In [9]:
sum(output_df["CourseTaken"] == 1)

895